<a href="https://colab.research.google.com/github/alexlautw9527/for_notebook/blob/main/%E7%8E%89%E5%B1%B12021%E5%86%AC%E5%AD%A3%E8%B3%BD%E6%8E%A8%E8%96%A6%E7%B3%BB%E7%B5%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd 
import numpy as np

In [ ]:
%%capture
!python -m pip install "dask[complete]" 

In [6]:
!cp /content/drive/MyDrive/02_GNL_DATA/2021玉山人工智慧公開挑戰賽冬季賽訓練資料集.gz.gz /content

In [7]:
!gunzip /content/2021玉山人工智慧公開挑戰賽冬季賽訓練資料集.gz.gz
!tar -xvf /content/2021玉山人工智慧公開挑戰賽冬季賽訓練資料集.gz

In [12]:
import dask.dataframe as dd
df = dd.read_csv('/content/tbrain_cc_training_48tags_hash_final.csv',assume_missing=True)

In [13]:
%%time
## 把csv切成32個parquet格式
df = df.repartition(npartitions=32) 
df = dd.to_parquet(df,path='/content/data.parquet', engine='pyarrow')

CPU times: user 4min 49s, sys: 14.7 s, total: 5min 3s
Wall time: 1min 28s


In [14]:
## 載入切好格式

df = dd.read_parquet('data.parquet', engine='pyarrow')

In [15]:
%%time
##看看一個id都幾筆資料
df.groupby('chid').size().describe().compute()

CPU times: user 38.6 s, sys: 18.1 s, total: 56.7 s
Wall time: 18.2 s


count    500000.000000
mean         65.951306
std          55.929864
min           1.000000
25%          24.000000
50%          50.000000
75%          93.000000
max         441.000000
dtype: float64

In [18]:
df.head()

,dt,chid,shop_tag,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,domestic_offline_amt_pct,domestic_online_amt_pct,overseas_offline_amt_pct,overseas_online_amt_pct,card_1_txn_cnt,card_2_txn_cnt,card_3_txn_cnt,card_4_txn_cnt,card_5_txn_cnt,card_6_txn_cnt,card_7_txn_cnt,card_8_txn_cnt,card_9_txn_cnt,card_10_txn_cnt,card_11_txn_cnt,card_12_txn_cnt,card_13_txn_cnt,card_14_txn_cnt,card_other_txn_cnt,card_1_txn_amt_pct,card_2_txn_amt_pct,card_3_txn_amt_pct,card_4_txn_amt_pct,card_5_txn_amt_pct,card_6_txn_amt_pct,card_7_txn_amt_pct,card_8_txn_amt_pct,card_9_txn_amt_pct,card_10_txn_amt_pct,card_11_txn_amt_pct,card_12_txn_amt_pct,card_13_txn_amt_pct,card_14_txn_amt_pct,card_other_txn_amt_pct,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1.0,10321418.0,45,3.0,3891.965283,3.0,0.0,0.0,0.0,1.00,0.00,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,5.0,1.0,99.0,30.0,95982.822967,1.0,4.0,1.0
1,1.0,10414574.0,15,2.0,10616.561549,2.0,0.0,0.0,0.0,1.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,15.0,1.0,2.0,30.0,130702.351368,1.0,3.0,1.0
2,1.0,10134567.0,48,2.0,23527.655416,0.0,2.0,0.0,0.0,0.00,1.00,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,11.0,1.0,3.0,30.0,112010.611717,0.0,4.0,1.0
3,1.0,10001003.0,48,9.0,17751.558260,7.0,2.0,0.0,0.0,0.75,0.25,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,11.0,1.0,2.0,30.0,59701.507360,0.0,3.0,1.0
4,1.0,10267183.0,2,1.0,21701.307598,1.0,0.0,0.0,0.0,1.00,0.00,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,6.0,15.0,1.0,99.0,30.0,NaN,0.0,6.0,0.0


In [19]:
%%time

## 計算by id各類別總計消費金額
newdf = df.groupby(['chid','shop_tag'])['txn_amt'].sum().compute().unstack(fill_value=0)

CPU times: user 1min 11s, sys: 20.4 s, total: 1min 32s
Wall time: 34.9 s


In [63]:
newdf.head()

shop_tag,1,10,11,12,13,14,15,16,17,18,19,2,20,21,22,23,24,25,26,27,28,29,3,30,31,32,33,34,35,36,37,38,39,4,40,41,42,43,44,45,46,47,48,5,6,7,8,9,other
chid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10000000.0,0.0,191770.225706,0.0,76268.969509,0.0,0.0,4684.483372,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,4235.791755,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,62218.660703,81337.066461,0.0,9156.086033,0.0,0.00000,0.0,0.0,0.000000,40746.482820
10000001.0,0.0,12847.384866,0.0,44104.144043,0.0,0.0,57821.590353,12643.832959,0.000000,8019.906666,0.0,17649.573492,0.0,25146.943067,3945.661088,0.0,0.0,0.0,39478.512075,0.0,0.0,0.0,6308.563642,0.0,595.018412,0.0,0.0,51329.246565,0.0,26998.210065,56105.751640,0.0,49331.267298,0.0,18721.636466,0.0,14317.563657,0.0,6048.351042,63374.858212,0.000000,0.0,84246.368876,0.0,26023.22562,0.0,0.0,0.000000,50488.365846
10000002.0,0.0,16755.695323,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,3571.095195,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,5692.923612,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.00000,0.0,0.0,0.000000,37649.183244
10000003.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,8554.030804,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,8023.582985,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.00000,0.0,0.0,0.000000,0.000000
10000004.0,0.0,0.000000,0.0,15271.878083,0.0,0.0,5773.036485,0.000000,6037.642859,0.000000,0.0,10825.822388,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,12617.127283,0.0,0.0,0.000000,0.0,11546.072969,34238.290641,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,19790.314068,0.000000,0.0,5419.225995,0.0,0.00000,0.0,0.0,10659.532107,12386.980405


In [28]:
np.argmax(newdf,axis=1)

array([ 1, 42, 48, ..., 35, 39, 14])

In [64]:

## 取topn最大值的位置 (為了直接映射colname, 而colname即是類別)
n=0
(-newdf.values).argsort(axis=1)[:,n]

array([ 1, 42, 48, ..., 35, 39, 14])

In [40]:
top1 = [list(newdf.columns)[x] for x in (-newdf.values).argsort(axis=1)[:,0]]
top2 = [list(newdf.columns)[x] for x in (-newdf.values).argsort(axis=1)[:,1]]
top3 = [list(newdf.columns)[x] for x in (-newdf.values).argsort(axis=1)[:,2]]

In [43]:
top_df = pd.DataFrame({'chid':newdf.index, 'top1':top1, 'top2':top2, 'top3':top3})
top_df['chid'] = top_df['chid'].astype(int)

In [46]:
## 消費類別在這些list才是預測範圍

pred_list = ['2',
'6',
'10',
'12',
'13',
'15',
'18',
'19',
'21',
'22',
'25',
'26',
'36',
'37',
'39',
'48']

In [48]:
top_df['top1'] = np.where(top_df['top1'].isin(pred_list), top_df['top1'], np.nan)
top_df['top2'] = np.where(top_df['top2'].isin(pred_list), top_df['top2'], np.nan)
top_df['top3'] = np.where(top_df['top3'].isin(pred_list), top_df['top3'], np.nan)

37 15 36為先驗top3最多的消費類別

In [60]:
top_df = top_df.fillna('37')

In [61]:
top_df.to_csv('submit2.csv', index=False)

In [62]:
top_df

,chid,top1,top2,top3
0,10000000,10,37,12
1,10000001,48,37,15
2,10000002,37,10,37
3,10000003,22,37,37
4,10000004,37,37,12
...,...,...,...,...
499995,10499995,2,10,37
499996,10499996,48,15,37
499997,10499997,37,25,37
499998,10499998,37,39,37
